# Test Data Generation

- generating good test data can be challenging
- use Hypothesis library - https://hypothesis.readthedocs.io/en/latest/index.html
- Hypothesis is a Python library for creating unit tests by automatically generating meaningful test data
    - helps create edge test cases in your code, you'd not have thought to look for
    - can use it with pytest and unittest libraries
- hypothesis provides property-based testing 
- designed to test the aspects of a data property that should always be true
- allows for a range of inputs to be programmed and tested within a single test, rather than having to write a different test (hard-coded inputs) for every value that you want to test
- let's you do **fuzz testing**
    - an automated software testing method that injects invalid, malformed, or unexpected inputs into a system to reveal software defects and vulnerabilities 
- install hypothesis library
- more detail examples: https://towardsdatascience.com/automating-unit-tests-in-python-with-hypothesis-d53affdc1eba

```bash
pip install hypothesis
```
- see what data can you generate and how docs: https://hypothesis.readthedocs.io/en/latest/data.html#

In [1]:
! pip install hypothesis

In [2]:
def add(nums:list[int]) -> int:
    s = 0
    for n in nums:
        s += n
    return s

In [3]:
# typical unittesting; hardcoded input provided to functions/methods provides expected output
assert add([1, 2, 3]) == 6, '1 2 3 did NOT add to 6'
assert add([1, 3, -1, 0, -1]) == 2, '1, 3, -1, 0, -1 did NOT add to 2'
print('all tests done...')

all tests done...


In [4]:
# see settings docs: https://hypothesis.readthedocs.io/en/latest/settings.html
from hypothesis import given, settings, Verbosity
import hypothesis.strategies as some

In [9]:
# by default generates 200 random lists of integers
@given(some.lists(some.integers()))
def test_add(nums):
    #print(nums) # uncomment it to see what nums are generated
    assert add(nums) == sum(nums)

In [10]:
test_add()

In [20]:
# more examples...
@given(some.integers(), some.integers())
# can set it to control the no. of examples, database, randomization, etc.
@settings(max_examples=100, verbosity=Verbosity.verbose, derandomize=True) 
def test_ints_are_commutative(x, y):
    assert x + y == y + x

In [21]:
test_ints_are_commutative()

Trying example: test_ints_are_commutative(
    x=0,
    y=0,
)
Trying example: test_ints_are_commutative(
    x=0,
    y=0,
)
Trying example: test_ints_are_commutative(
    x=-75595815,
    y=25546,
)
Trying example: test_ints_are_commutative(
    x=0,
    y=0,
)
Trying example: test_ints_are_commutative(
    x=-28065,
    y=-79,
)
Trying example: test_ints_are_commutative(
    x=0,
    y=0,
)
Trying example: test_ints_are_commutative(
    x=-4559,
    y=4665598009820062891,
)
Trying example: test_ints_are_commutative(
    x=0,
    y=0,
)
Trying example: test_ints_are_commutative(
    x=14153,
    y=17605,
)
Trying example: test_ints_are_commutative(
    x=0,
    y=0,
)
Trying example: test_ints_are_commutative(
    x=0,
    y=0,
)
Trying example: test_ints_are_commutative(
    x=-3375,
    y=1394,
)
Trying example: test_ints_are_commutative(
    x=1394,
    y=1394,
)
Trying example: test_ints_are_commutative(
    x=1394,
    y=1394,
)
Trying example: test_ints_are_commutative(
    x=-

In [18]:
# explicitly give name to data
@given(x=some.integers(), y=some.integers())
def test_ints_cancel(x, y):
    assert (x + y) - y == x

In [10]:
test_ints_cancel()

In [11]:
# generate lists of arbitrary length (usually between 0 and
# 100 elements) whose elements are integers.
@given(some.lists(some.integers()))
def test_reversing_twice_gives_same_list(xs):
    ys = list(xs)
    ys.reverse()
    ys.reverse()
    assert ys == xs 

In [12]:
test_reversing_twice_gives_same_list()

In [13]:
@given(some.tuples(some.booleans(), some.text()))
def test_look_tuples_work_too(t):
    # A tuple is generated as the one you provided, 
    # with the corresponding types in those positions.
    assert len(t) == 2
    assert isinstance(t[0], bool)
    assert isinstance(t[1], str)

In [14]:
# generate even numbers between 10 and 20
# use min_value and max_value or map method
@given(some.integers(min_value=5, max_value=10).map(lambda x: x*2))
def test_somefunc(num):
    print(num)
    #assert test some functions using nums!

In [15]:
test_somefunc()

10
10
10
14
20
16
12
18


In [16]:
# can compose types...
# list with at most 100 integers with min value of 1
@given(some.lists(some.integers(min_value=1), max_size=100))
def test_func1(nums):
    print(nums)
    

In [17]:
test_func1()

[]
[1]
[]
[1]
[1]
[7146]
[1]
[1]
[1]
[1]
[1]
[16455, 441228481, 6823]
[7817, 24988, 834970296, 3772, 6176, 30179, 26507, 6953042308341101676]
[17483, 7, 108, 2, 97, 6214, 19013, 17449]
[81, 24, 55, 92, 22634582747798802293469837499090898669, 1891237215, 12299, 22114, 2022325074, 3577, 17553, 20, 19263, 2, 20920, 79, 86, 21, 5006, 4545, 2, 17228, 4431, 122380935052757737661683989782079054897, 107]
[1013610165, 84, 11, 22846, 15149, 2133525608853353919, 9022, 68]
[110, 1875753942, 31253, 3359, 73, 15109, 9700, 71, 13837, 961893165, 18471, 13, 32271, 15, 23543, 2039830745, 95, 23092, 15]
[3504465422682665793, 2081078129409907908]
[2081078129409907908, 2081078129409907908]
[29762, 3950, 25211, 4859]
[21956, 5507, 26991, 6, 2308, 19704]
[2308, 5507, 26991, 6, 2308, 19704]
[8619]
[8619, 8619, 20]
[24216, 2882, 25933, 122, 13025, 3853]
[55, 618954289, 29862, 96, 12609, 69]
[55]
[16763, 11558]
[12, 1165, 1434]
[6, 4]
[6]
[5658, 19659, 98, 25, 14]
[26691, 2828, 5254]
[129, 2, 5254]
[129, 2, 525

In [26]:
# define a function that takes an integer value between 1 and 10 and returns the square root of the value
def int_sqrt(n: int) -> float:
    # is this correct implementation?
    return n**0.5

In [27]:
def test_int_sqrt():
    assert int_sqrt(9) == 3, 'sqrt(9) != 3'
    assert int_sqrt(4) == 2, 'sqrt(4) != 2'
    #assert int_sqrt(100) == 10, 'sqrt(100) != 10'
    # any problem here...?
    print('all test PASS...')

In [28]:
test_int_sqrt()

all test PASS...


In [29]:
# property-based testing using hypothesis
from dataclasses import dataclass
import hypothesis.strategies as st

@dataclass
class TestData:
    int_value: st.SearchStrategy[int]

# generating correct input data range
test_data = TestData(int_value=st.integers(min_value=1, max_value=10))

In [30]:
@given(st.data())
def test_int_sqrt(data: st.DataObject):
    import math

    an_int = data.draw(test_data.int_value)
    root = int_sqrt(an_int)
    #print(an_int, root) # uncomment to see the test data

    assert isinstance(an_int, int)
    assert root == math.sqrt(an_int)
    print('all answer correct')

In [31]:
test_int_sqrt()

all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct
all answer correct


In [32]:
# what if you pass string, -negative, 0, float, larger than 10 values...

# let's test for -ve values
@given(some.integers(min_value=-100000, max_value=-1))
def test_int_sqrt_negative(n: int):
    # this should throw AssertionError, but does it..
    try:
        #print(n)
        root = int_sqrt(n)
    except AssertionError:
        # this must be printed... to pass the test
        print('assertion error thrown...PASS')
    else:
        print('FAIL')

In [33]:
test_int_sqrt_negative()

FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL


In [34]:
# let's test for larger than 10 values
@given(some.integers(min_value=11, max_value=100))
def test_int_sqrt_larger_positives(n: int):
    # this should throw AssertionError, but does it..
    try:
        #print(n)
        root = int_sqrt(n)
    except AssertionError:
        # this must be printed... to pass the test
        print('assertion error thrown...PASS')
    else:
        print('FAIL')

In [35]:
test_int_sqrt_larger_positives()

FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL


In [36]:
# let's test with float values
@given(some.floats())
def test_int_sqrt_floats(n: float):
    # this should throw AssertionError, but does it..
    try:
        #print(n)
        root = int_sqrt(n)
    except AssertionError:
        # this must be printed... to pass the test
        print('assertion error thrown...PASS')
    else:
        print('FAIL')

In [37]:
test_int_sqrt_floats()

FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL
FAIL


In [38]:
# let's test with some strings
@given(some.text())
def test_int_sqrt_strings(n: str):
    # this should throw AssertionError, but does it..
    try:
        #print(n)
        root = int_sqrt(n)
    except AssertionError:
        # this must be printed... to pass the test
        print('assertion error thrown...PASS')
    else:
        print('FAIL')

In [39]:
test_int_sqrt_strings()

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'float'

## Fix int_sqrt( ) so all property-based test PASS

- since the test uses AssertionError exception, use assert to fix
- you could decide to throw your own custom error for invalid data and assert those errors accordingly

## property-based testing demo

- see `src/unittesting/inventory` folder
    - a simple order processing and stock control system
    - burrowed from book "The Pragmatic Programmer" by David Thomas and Andrew Hunt
- two classes:
    - `Warehouse` and `Order` in two separate modules
- run several test modules provided in the order:

```bash
pytest test_order.py
pytest test_order1.py # <-- this property-based testing using hypothesis will find error
pytest test_warehouse.py
pytest test_order_fixed.py
```

- performs several property-based tests
- automatically generates test data using hyposthesis
- finds the data that cause tests to fail
    - use the data to create the separate explicit unittest - which becomes your regression test
    - since the data is generated randomly, you may not guarantee the same data will be generated
- property-based tests often surprise you!

### regression test
- focus on the subset of unit tests targeting subset of new code/feature
- a type of software testing technique that re-runs functional and non-functional tests to ensure that a software application works as intended after any code changes, updates, revisions, improvements, or optimizations
- change int_sqrt( ) function to accept values from 0 to 100
- see if the existing test passes
    - do you need new proerty-based tests?